In [4]:
from generate_recipe_task import generate_task_file_from_df

generate_task_file_from_df('csv/cake_recipes_formatted.csv', 3)

["1 1/2 lb. ground beef", "1 c. tomato juice", "3/4 c. oats (uncooked)", "1 egg, beaten", "1/4 c. chopped onion", "1/4 tsp. pepper", "1 1/2 tsp. salt"]
["Mix well.", "Press firmly into an 8 1/2 x 4 1/2 x 2 1/2-inch loaf pan.", "Bake in preheated moderate oven.", "Bake at 350\u00b0 for 1 hour.", "Let stand 5 minutes before slicing.", "Makes 8 servings."]


KeyboardInterrupt: 